# 01 Preprocessing

This notebook covers the preprocessing of the 16S rRNA amplicon sequencing data. It includes data import, quality control, denoising, and taxonomic classification.

<img src=./figures/workflow_preprocessing.jpg alt="Description" width="750" height="">

## Setup

Activate the environment `microbEvolve` before running this Jupiter notebook. You can also submit this notebook as a SLURM job with the following command. Please make sure you submit the job from the `scrips/` directory.

```bash
sbatch --time=03:59:00 --cpus-per-task=16 --mem-per-cpu=6G --output=slurm-%j.out --error=slurm-%j.err --wrap="bash -c 'module load eth_proxy && source $HOME/.bashrc && conda activate microbEvolve && jupyter execute --inplace ./01-1_preprocessing.ipynb'"
```

This step loads all requited packages. The paths to the scripts and the data are stored in the variables `scripts_dir` and `data_dir`. 

In [1]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
scripts_dir = "src"
data_dir = "../data"

## Importing sequencing data and metadata

This step imports the demultiplexed data, which is already available as a QIIME 2 artifact on Polybox.

Next, the metadata is imported. It is provided as an Excel file with three sheets: DataDictionary, metadata_per_sample, and metadata_per_age. Each sheet is imported as its own .tsv file (`metadata_dictionary`, `metadata_per_sample`, and `metadata_per_age`). Lastly, the first column in metadata_per_sample `Unnamed: 0` is renamed to `sampleid`.

In [3]:
! bash {scripts_dir}/importing.sh

[2025-12-17 16:52:27] Starting importing script
[2025-12-17 16:52:27] Importing 16S rRNA sequencing data...
--2025-12-17 16:52:27--  https://polybox.ethz.ch/index.php/s/zi5ZBrBwcn7SYof/download/demux-paired-end.qza


Resolving proxy.service.consul (proxy.service.consul)... 10.205.212.167
Connecting to proxy.service.consul (proxy.service.consul)|10.205.212.167|:3128... connected.
Proxy request sent, awaiting response... 

200 OK
Length: 572079812 (546M) [application/octet-stream]
Saving to: ‘../data/raw/demux_paired_end.qza’

          ../data/r   0%[                    ]       0  --.-KB/s               

         ../data/ra  10%[=>                  ]  54.80M   274MB/s               

        ../data/raw  23%[===>                ] 126.71M   317MB/s               

...

  ../data/raw/demux  88%[================>   ] 481.94M   295MB/s               

../data/raw/demux_p 100%[===================>] 545.58M   302MB/s    in 1.8s    

2025-12-17 16:52:28 (302 MB/s) - ‘../data/raw/demux_paired_end.qza’ saved [572079812/572079812]



[2025-12-17 16:52:29] 16S rRNA sequencing import completed and stored in ../data/raw/dada2_rep_set.qza
[2025-12-17 16:52:29] Importing metadata...
--2025-12-17 16:52:29--  https://polybox.ethz.ch/index.php/s/YQQggAqcQCApJmQ/download
Resolving proxy.service.consul (proxy.service.consul)... 10.205.212.167
Connecting to proxy.service.consul (proxy.service.consul)|10.205.212.167|:3128... connected.
Proxy request sent, awaiting response... 

200 OK
Length: 25082 (24K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘../data/raw/metadata.xlsx’

../data/raw/metadat 100%[===================>]  24.49K  --.-KB/s    in 0.008s  

2025-12-17 16:52:30 (2.97 MB/s) - ‘../data/raw/metadata.xlsx’ saved [25082/25082]

[2025-12-17 16:52:30] Metadata import completed and stored in ../data/raw/metadata.xlsx
[2025-12-17 16:52:30] Saving metadata as single .tsv files...


[2025-12-17 16:52:31] metadata_dictionary, metadata_per_age and metadata_per_sample stored successfuly as single .tsv files in ../data/raw/
[2025-12-17 16:52:31] Import script completed successfully!


## Quality Control

For quality control, we converted the `demux_paired_end.qza` file into a `.qzv` artifact.

In [4]:
! bash {scripts_dir}/quality_control.sh

/cluster/home/fschindele/miniconda3/envs/microbEvolve/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Saved Visualization to: ../data/processed/demux_paired_end.qzv


In [5]:
Visualization.load(f"{data_dir}/processed/demux_paired_end.qzv")

<visualization: Visualization uuid: 7dcfb0da-abec-4b3a-a3a5-76661e04bca2>

The minimum and maximum sequencing length during subsampling was 301 bp for both forward and reverse reads.

Overall, sequence quality was very high:
- Forward reads: median quality score of 34 across all base positions.
- Reverse reads: median quality score of 34 up to position 293, dropping to 20 from position 294 onward.

Variability was generally higher in the reverse reads and increased substantially from position 221 onward.

## Cutadapt and Denoising

### Cutadapt

Interestingly, the V4 region of the 16S rRNA gene is much shorter than 301 bp, which is why we thought that we might still have primer sequences in our reads.

#### 1. Initial Trimming Attempt (Original and Modified Primers)

The first attempt used the known V4 specific forward and reverse primer sequences ([source](https://earthmicrobiome.ucsd.edu/protocols-and-standards/16s/)). We used the `--p-discard-untrimmed True` flag in order to see how many reads would be trimmed.
- This step resulted in zero sequences being trimmed or retained.
- In hindsight, it is irrelevant if the original or modified primers are used, as they are not required to match perfectly and the one base difference does not influence the result.
- This indicated that the forward and reverse primers were already removed from the sequences by the sequencing facility prior to data delivery. This was also confirmed by our TA. Because we anchored the primers, the `--p-discard-untrimmed True` setting caused all reads to be discarded even if the reverse complement might be present.

The other possibility that could explain the length of our reads would be read-through.

#### 2. Identifying and Trimming Read-Through (Successful Strategy)

To be able to identify read-through, even though the forward and reverse primers had already been removed from the sequences, we decided to only look for the reverse complement of the primers.
- Approximately 4.5 million sequences were successfully truncated and retained using this approach, which confirms the presence of read-through.
- The successful truncation yielded reads with an approximate length of 250 bases, which likely represents the true length of the amplicon.
- **Many reverse reads were longer** than 250 bases, suggesting that the low base quality towards the end prevented cutadapt from recognizing the reverse-complement forward primer due to many mismatches.

In [6]:
! bash {scripts_dir}/cutadapt.sh

/cluster/home/fschindele/miniconda3/envs/microbEvolve/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Saved SampleData[PairedEndSequencesWithQuality] to: ../data/raw/demux_paired_end_trimmed-modified-primers.qza


/cluster/home/fschindele/miniconda3/envs/microbEvolve/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Saved Visualization to: ../data/processed/demux_paired_end_trimmed-modified-primers.qzv


/cluster/home/fschindele/miniconda3/envs/microbEvolve/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Saved SampleData[PairedEndSequencesWithQuality] to: ../data/raw/demux_paired_end_trimmed-original-primers.qza


/cluster/home/fschindele/miniconda3/envs/microbEvolve/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Saved Visualization to: ../data/processed/demux_paired_end_trimmed-original-primers.qzv


In [7]:
Visualization.load(f"{data_dir}/processed/demux_paired_end_trimmed-modified-primers.qzv")

<visualization: Visualization uuid: 70a36933-f994-47f5-ba73-646eaaf866c9>

In [8]:
Visualization.load(f"{data_dir}/processed/demux_paired_end_trimmed-original-primers.qzv")

<visualization: Visualization uuid: 1c14c085-78f8-4d9c-96b4-a360321c758c>

### Denoising with DADA2

As trimming with cutadapt failed for many reverse reads, we decided to truncate aggressively during denoising to remove any read-through sequences. The truncation lengths were set to 220 bp for forward reads and 200 bp for reverse reads.

A truncation length of 220 basses for the forward reads and 200 bass pairs for the reverse reads resulted in good denoising performance. Around 90% of the reads passed the filtering step and nearly all of those reads were able to be merged.

In [9]:
! bash {scripts_dir}/denoising.sh

/cluster/home/fschindele/miniconda3/envs/microbEvolve/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Saved FeatureTable[Frequency] to: ../data/raw/dada2_table.qza


Saved FeatureData[Sequence] to: ../data/raw/dada2_rep_set.qza
Saved SampleData[DADA2Stats] to: ../data/raw/dada2_stats.qza


/cluster/home/fschindele/miniconda3/envs/microbEvolve/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Saved Visualization to: ../data/processed/dada2_stats.qzv


/cluster/home/fschindele/miniconda3/envs/microbEvolve/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Saved Visualization to: ../data/processed/dada2_rep_set.qzv


/cluster/home/fschindele/miniconda3/envs/microbEvolve/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Saved Visualization to: ../data/processed/dada2_table.qzv


In [10]:
Visualization.load(f"{data_dir}/processed/dada2_stats.qzv")

<visualization: Visualization uuid: 84f0bfb0-000d-497e-9e87-147382d41d28>

## Taxonomy

To identify which organisms are present in our samples, we used a pretrained, weighted classifier optimized for stool samples. It targets the 16S rRNA V4 region (515F/806R) and is based on the SILVA 138.2 database (99% NR). This classifier incorporates weights derived from a large database of human stool samples. This is designed to improve classification accuracy for samples derived from the human gut by prioritizing taxa commonly found in that environment.

In [11]:
! ./$scripts_dir/taxonomy.sh

[2025-12-17 17:13:50] Starting taxonomy classification script


[2025-12-17 17:13:50] Input file verified: ../data/raw/dada2_rep_set.qza
[2025-12-17 17:13:50] Downloading weighted classifier for human stool samples...
--2025-12-17 17:13:50--  https://www.arb-silva.de/fileadmin/silva_databases/current/QIIME2/2025.7/SSU/V4-515f-806r/weighted/human-stool/SILVA138.2_SSURef_NR99_weighted_classifier_V4-515f-806r_human-stool.qza
Resolving proxy.service.consul (proxy.service.consul)... 10.205.212.167
Connecting to proxy.service.consul (proxy.service.consul)|10.205.212.167|:3128... connected.


Proxy request sent, awaiting response... 

200 OK
Length: 164259156 (157M) [application/octet-stream]
Saving to: ‘../data/raw/silva-138-99-515-806-nb-classifier-weighted-stool.qza’

          ../data/r   0%[                    ]       0  --.-KB/s               

         ../data/ra   0%[                    ]   1.30M  5.83MB/s               

...

Saved Visualization to: ../data/processed/taxonomy_weighted_stool.qzv


[2025-12-17 17:19:30] Weighted taxonomy visualization created
[2025-12-17 17:19:30] Creating taxa bar plot for weighted taxonomy results...


/cluster/home/fschindele/miniconda3/envs/microbEvolve/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Saved Visualization to: ../data/processed/taxa-bar-plots_weighted.qzv


[2025-12-17 17:20:04] Taxa bar plot for weighted taxonomy results created
[2025-12-17 17:20:04] Taxonomy classification script completed successfully!
